In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Função para Agrupar CPP + ConsumoGás

In [2]:
def correcao_varredura_consumo(df_cpp, df_consumo):
    """
        Funcao responsavel por agrupar o consumo no turno
        Arguments:
          df_cpp: dataframe -- Dataframe com as colunas: Inicio, Fim, Turno e Produção
        Return:
          Time_i: list -- 
          Time_f: list -- 
          consumo_: list -- 
          produ__: list -- 
          tur__: list -- 
    """
    Time_i = []
    Time_f = []
    consumo_ = []
    produ__ = []
    tur__ = []
    cod_produto = []

    for data_, turno, prod_, cod_prod in zip(df_cpp.DATA,df_cpp.TURNO, df_cpp['PRODUÇÃO REAL (kg)'], 
                                             df_cpp['CÓDIGO DO PRODUTO']):
        if turno == 'A':
            h_inicio = ' 06:00:00'
            h_fim = ' 13:59:59'
    
        elif turno == 'B':
            h_inicio = ' 14:00:00'
            h_fim = ' 21:59:59'
    
        else:
            h_inicio = ' 22:00:00'
            h_fim = ' 05:59:59'

        inicio = data_ + h_inicio
        fim = data_ + h_fim
    
        mask = (df_consumo['Time'] >= inicio) & (df_consumo['Time'] <= fim) # Agrupando datas de um intervalo
        grupo = df_consumo.loc[mask]
        
        teste_2 = []
        for j in grupo.consumo: # sempre mudar
            try:
                j = j.split(' m³')[0]
                teste_2.append(float(j))
            except:
                teste_2.append(float(j))
        
        grupo.consumo = teste_2
    
        Time_i.append(inicio)
        Time_f.append(fim)
        consumo_.append(np.sum(grupo.consumo))
        produ__.append(prod_)
        tur__.append(turno)
        cod_produto.append(cod_prod)
    

    return Time_i, Time_f, consumo_, produ__, tur__, cod_produto

# Arquivo de Produção

In [3]:
with open(r'C:\Users\vitor\Documents\M_Dias\cpp_filtrada_linha\cpp_09_abril_a_novembro.csv', encoding='utf-8') as g:
    df_cpp = pd.read_csv(g, sep=',')

    
# df_cpp.info()
# df_cpp.head(2)


# Arquivo de consumo de gás

In [4]:
with open(r'C:\Users\vitor\Documents\M_Dias\consumo_gas\Consumo_gas_Linha_09_stone.csv', encoding='utf-8') as f:
    df_con = pd.read_csv(f, sep=',')
#     df_con.info()
    df_con['time'] = df_con['Time'].str.replace('-', '/')
    df_con = df_con.dropna()
    df_con[['date', 'hora']] = df_con['time'].str.split(expand=True)
    df_con[['ano', 'mes', 'dia']] = df_con['date'].str.split('/', expand=True)
    df_con['Data1'] = df_con['dia'].map(str) + '/' + df_con['mes'].map(str) + '/' + df_con['ano'].map(str) + ' ' + df_con['hora'].map(str)
    df_con = df_con[['Data1', 'Fábrica 2 > Linha 09 de Biscoitos > Forno']]
    df_con = df_con.rename(columns={'Data1': 'Time', 'Fábrica 2 > Linha 09 de Biscoitos > Forno': 'consumo'})


# print('---///---'*6)
# display(df_con.head(2))
# print('---///---'*6)
# print(df_con.info())



# Criando um Dataframe unido

In [5]:
Time_i, Time_f, consumo_, produ__, tur__, cod_produto = correcao_varredura_consumo(df_cpp, df_con)

data_ = {'Time_inicio': Time_i,
          'Time_fim': Time_f,
            'Producao_Kg': produ__,
            'consumo_Nm³': consumo_,
            'Turno': tur__,
             'cod.Produto': cod_produto}
new_df = pd.DataFrame(data_)

In [6]:
# new_df

# Salvando o arquivo em csv

In [7]:
# new_df.to_csv(r'C:\Users\vitor\Documents\M_dias\Análises_jaboatao\Tabelas_p_enviar\consumo_l09JB.csv', index=False, encoding='utf-8')